# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import datetime
import time
import dateutil.parser
import requests
from typing import List, Dict

%matplotlib inline
plt.rcParams['figure.figsize'] = [17, 7]

---
# Part 1. test Order Book Data

### Get Order Book Data

In [2]:
def orderbook_data(depth: int, contract: str = "ETHUSD", raw_data=False):

    url = "https://www.bitmex.com/api/v1/orderBook/L2"
    book_data = {'buys': 0, 'sells': 0}
    raw_book_data = {'total_buys': 0, 'sum_of_buys': 0, 'furthest_5_buy_prices': [],
                    'total_sells': 0, 'sum_of_sells': 0, 'furthest_5_sell_prices': []}


    # Set inputs
    data = dict()
    data['symbol'] = contract
    data['depth'] = depth

    try:
        response = requests.get(url, params=data)
    except Exception as e:
        print(f"Connecton error while making GET request to {url}: {e}")
        return

    if response.status_code == 200:
        response_data = response.json()
    else:
        print(f"Error while making GET request to {url}: {response.status_code}")
        print(response.headers)
        return None
    
    if response_data is not None:
        if raw_data:
            for idx, data in enumerate(response_data):

                if idx < 5:
                    raw_book_data['furthest_5_sell_prices'].append(data['price'])

                if idx > len(response_data)-6:
                    raw_book_data['furthest_5_buy_prices'].append(data['price'])

                if data['side'] == "Buy":
                    raw_book_data["sum_of_buys"] += data["size"]
                    raw_book_data['total_buys'] += 1

                elif data['side'] == "Sell":
                    raw_book_data["sum_of_sells"] += data["size"]
                    raw_book_data['total_sells'] += 1

            return raw_book_data

        else:
            for data in response_data:
                if data['side'] == "Buy":
                    book_data["buys"] += data["size"]

                elif data['side'] == "Sell":
                    book_data["sells"] += data["size"]
    
            return book_data

### Get current price

In [3]:
def get_curr_price(contract: str) -> str:

    url = "https://www.bitmex.com/api/v1/trade/bucketed"

    data = dict()
    data['symbol'] = contract
    data['partial'] = True  # returns a candle if it is not finished yet
    data['binSize'] = '1m'
    data['count'] = 1   # how many candles we can return (500 max)
    data['reverse'] = True

    try:
        response = requests.get(url, params=data)
    except Exception as e:
        print(f"Connecton error while making GET request to {url}: {e}")
        return

    if response.status_code == 200:
        raw_candles = response.json()
    else:
        print(f"Error while making GET request to {url}: {response.status_code}")
        print(response.headers)
        return None
    
    if raw_candles is not None:
        return raw_candles[0]['close']

### Get Size of the OrderBook

Run some stats on the entire orderbook

In [4]:
contract = "XBTUSD"

book_size = orderbook_data(0, contract, raw_data=True)
book_size_avg = (book_size["total_buys"] + book_size["total_sells"]) // 2

curr_price = get_curr_price(contract)

ret = "Buyers" if book_size["total_buys"] > book_size["total_sells"] else "sellers"

if ret == "Buyers":
    amount = book_size["total_buys"] - book_size["total_sells"]
else:
    amount = book_size["total_sells"] - book_size["total_buys"]


print(f"\nCurrent {contract} price: ${curr_price:,.2f}")

print(f"\nThe average size of the order book for {contract} is: {book_size_avg}")
print(f"There are {amount} more orders from {ret}")

print(f'\nTotal orderbook size: {book_size["total_buys"]+book_size["total_sells"]}')
print(f'Total buys: {book_size["total_buys"]}')
print(f'Total sells: {book_size["total_sells"]}')

print(f"\nTotal buy orderbook volume: ${book_size['sum_of_buys']:,.2f}")
print(f"Total sell orderbook volume: ${book_size['sum_of_sells']:,.2f}")

print('\nFurthest 5 buy prices:')
for i, v in enumerate(book_size['furthest_5_buy_prices']):
    print(f"{i+1} => ${v:,.2f}")

print('\nFurthest 5 sell prices:')
for i, v in enumerate(book_size['furthest_5_sell_prices'][::-1]):
    print(f"{i+1} => ${v:,.2f}")


Current XBTUSD price: $19,180.00

The average size of the order book for XBTUSD is: 2654
There are 1349 more orders from sellers

Total orderbook size: 5309
Total buys: 1980
Total sells: 3329

Total buy orderbook volume: $79,046,700.00
Total sell orderbook volume: $107,927,300.00

Furthest 5 buy prices:
1 => $30.00
2 => $10.00
3 => $1.50
4 => $1.00
5 => $0.50

Furthest 5 sell prices:
1 => $573,000.00
2 => $657,700.00
3 => $658,900.00
4 => $662,520.00
5 => $1,000,000.00


---
# Part 2. Get the OrderBook Data by Percentiles

### Get data

This time we will pass the variable $k$, a percentile to dictate how far from the current price we want to retrive order book data. 

In [5]:
def orderbook_data_by_percentile(k_buy: float, k_sell: float, contract: str = "ETHUSD"):

    url = "https://www.bitmex.com/api/v1/orderBook/L2"

    raw_book_data = {'total_buys': 0, 'sum_of_buys': 0, 'total_sells': 0, 'sum_of_sells': 0}


    # Set inputs
    data = dict()
    data['symbol'] = contract
    data['depth'] = 0

    try:
        response = requests.get(url, params=data)

    except Exception as e:
        print(f"Connecton error while making GET request to {url}: {e}")
        return

    if response.status_code == 200:
        response_data = response.json()

    else:
        print(f"Error while making GET request to {url}: {response.status_code}")
        print(response.headers)
        return None
    
    if response_data is not None:

        for idx, data in enumerate(response_data):

            if data['side'] == "Buy":
                if data['price'] > k_buy:
                    raw_book_data["sum_of_buys"] += data["size"]
                    raw_book_data['total_buys'] += 1

            elif data['side'] == "Sell":
                if data['price'] < k_sell:
                    raw_book_data["sum_of_sells"] += data["size"]
                    raw_book_data['total_sells'] += 1

        return raw_book_data


### Collect data for each $5%$ above and below the current price and store the results in a Pandas ```DataFrame```

In [6]:
curr_price = get_curr_price(contract)
df_data = []
percentile, idx = 0.05, 0

while percentile <= 0.5:
    k_buy = curr_price - (curr_price * percentile)
    k_sell = curr_price + (curr_price * percentile)

    book_data = orderbook_data_by_percentile(k_buy, k_sell, contract)

    df_data.append([f"{round(percentile*100)}%", book_data['total_buys'], book_data['total_sells'],
                f"${book_data['sum_of_buys']:,.2f}", f"${book_data['sum_of_sells']:,.2f}", f"${abs(book_data['sum_of_sells']-book_data['sum_of_buys']):,.2f}"])

    percentile += 0.05
    idx == 1

df = pd.DataFrame(df_data, columns=["%_from_current_price", "total_buy_orders", "total_sell_orders", "Buy_volume", "sell_volume", "Difference"])
df.set_index('%_from_current_price', inplace=True)

print(f"Current price: {curr_price}")
df

Current price: 19180


,total_buy_orders,total_sell_orders,Buy_volume,sell_volume,Difference
%_from_current_price,,,,,
5%,738,570,"$24,962,300.00","$20,919,800.00","$4,042,500.00"
10%,1019,830,"$30,313,600.00","$26,065,000.00","$4,248,600.00"
15%,1228,1067,"$39,244,600.00","$28,158,700.00","$11,085,900.00"
20%,1402,1234,"$45,522,000.00","$38,273,500.00","$7,248,500.00"
25%,1516,1328,"$51,564,900.00","$46,154,000.00","$5,410,900.00"
30%,1628,1414,"$57,890,100.00","$54,099,900.00","$3,790,200.00"
35%,1699,1490,"$65,782,100.00","$64,861,000.00","$921,100.00"
40%,1762,1562,"$76,688,400.00","$65,504,200.00","$11,184,200.00"
45%,1800,1661,"$77,070,000.00","$78,691,300.00","$1,621,300.00"
